# Iris with Artificial Neural Networks
ANNs are a class of ML models inspired by the structure and function of the human brain. In this notebook, we will implement ANN to solve popular **iris classification problem**.

### Load Iris dataset

In [1]:
import pandas as pd

iris_data = pd.read_csv('./datasets/iris.csv')
iris_data.head(5)

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [2]:
X = iris_data[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']].values
Y = iris_data['species'].values

In [3]:
X.shape

(150, 4)

In [4]:
Y.shape

(150,)

### Prepare data

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=32, shuffle=True)

Transform and scale your data

In [6]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [7]:
X_test[:3]

array([[-0.21239804, -0.55325422,  0.38855124,  0.0939925 ],
       [-1.54114406,  1.31690089, -1.6137987 , -1.34868301],
       [-1.05796369,  0.84936211, -1.27053871, -1.08637838]])

In [8]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
Y_train = encoder.fit_transform(Y_train)
Y_test = encoder.transform(Y_test)

In [9]:
import numpy as np

np.unique_values(Y_train)

array([1, 0, 2])

Convert to tensors

In [10]:
import torch

X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
Y_train = torch.tensor(Y_train, dtype=torch.long)
Y_test = torch.tensor(Y_test, dtype=torch.long)

In [11]:
from torch.utils.data import TensorDataset, DataLoader

train_dataset = TensorDataset(X_train, Y_train)
test_dataset = TensorDataset(X_test, Y_test)

train_loader = DataLoader(train_dataset, 32, True)
test_loader = DataLoader(test_dataset, 32, True)

### Create *ANN* architecture

In [12]:
import torch.nn as nn

class IrisANN(nn.Module):

    def __init__(self, in_size=4, hidden_size=32, out_size=3):
        super().__init__()

        self.layers = nn.Sequential(
            nn.Linear(in_size, hidden_size),
            nn.ReLU(),
            
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),

            nn.Linear(hidden_size, out_size)
        )
    
    def forward(self, x):
        return self.layers(x)

In [13]:
model = IrisANN()

### Training of *ANN*

In [14]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [15]:
n_epochs = 30

for _ in range(n_epochs):
    model.train()

    for X_batch, Y_batch in train_loader:
        optimizer.zero_grad()
        outputs = model(X_batch)
        
        loss = criterion(outputs, Y_batch)
        loss.backward()

        optimizer.step()


### Evaluation of *ANN*

In [16]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        outputs = model(X_batch)
        _, predicted = torch.max(outputs, 1)
        
        total += y_batch.size(0)
        correct += (predicted == y_batch).sum().item()

print(f"Test Accuracy: {correct/total * 100:.2f}%")

Test Accuracy: 90.00%
